# Data voorbewerking

De data voor dit project is afkomstig van het back office systeem van de Volksbank Financial Markets, deels aangevuld met extra data vanuit Bloomberg.
De data voor dit onderzoek bestaat uit de volgende data sets:

- Bond data
- Bondprijzen
- Government Yield curves
- Inflation data

Alle data is uit de bronsystemen geextraheerd en opgeslagen in csv files. In dit workbook lopen we door de data voorbereiding heen. Alle hier genoemde stappen kunnen ook geautomatiseerd in een keer worden uitgevoerd door het aanroepen van de make_data routine zoals te zien aan het eind van dit notebook.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.insert(0, "..") 
from src.data import make_dataset, join_data
from src.features import build_features

# Bond data

Bond data betreft de basis karakteristieken van de bond. Het rente percentage, de looptijd e.d.

In [2]:

# Get bond data, drop unneeded columns, convert formats and strip training blanks
df_bonds = make_dataset.get_bond_data()

2022-02-09 09:58:43.351 | INFO     | src.data.make_dataset:get_bond_data:31 - Load bond data
2022-02-09 09:58:43.353 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\bonds.csv


De issuer rating ontbreekt in sommige gevallen. Deze imputeren we met de meest voorkomende issue rating voor de issuer.
De CFI code (Product classificatie) wordt waar deze ontbreekt ingevuld met code 'onbekend' = DXXXXX. Datums met de waarde 1899-12-30 zijn default waardes van het bronsysteem. Deze worden verwijderd. Daar waar de eerste coupon datum ontbreekt (o.a. zero coupon bonds) - wordt deze aangevuld met de issue datum. De reden hiervoor is dat we op deze wijze zonder veel moeite de looptijd van de bond kunnen berekenen.

In [3]:
df_bonds = make_dataset.impute_bonds(df_bonds)

2022-02-09 09:58:43.515 | INFO     | src.data.make_dataset:impute_bonds:82 - Impute bond data


In [4]:
df_bonds.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 0 to 229
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ccy                226 non-null    string        
 1   country            226 non-null    string        
 2   bond_ext_name      226 non-null    string        
 3   issue_dt           226 non-null    datetime64[ns]
 4   first_coupon_date  226 non-null    datetime64[ns]
 5   mature_dt          226 non-null    datetime64[ns]
 6   isin               226 non-null    string        
 7   issuer_name        226 non-null    string        
 8   coupon_frq         226 non-null    string        
 9   coupon             226 non-null    float64       
 10  tot_issue          226 non-null    float64       
 11  cfi_code           226 non-null    string        
 12  issue_rating       226 non-null    string        
 13  bond_duration      226 non-null    int64         
 14  issue     

In [5]:
make_dataset.save_pkl('bonds', df_bonds)

2022-02-09 09:58:43.839 | INFO     | src.data.make_dataset:save_pkl:344 - Save preprocessed bonds data


# Bondprijzen

De bondprijzen zijn beschikbaar per bond per dag. Dit zijn gemiddelde eindedag prijzen van verschillende aanbieders. In het bronsysteem zijn de prijzen gemiddeld door de hoogste en laagste aanbieder uit te sluiten en de resterende prijzen te middelen. 

Bondprijzen worden boven of onder par (100) genoteerd. Een prijs boven 100 wil zeggen dat een extra premie betaald moet worden boven op de nominale waarde. Bij een prijs onder de 100 wordt een discount gerekend waardoor de bond aantrekkelijker geprijst is voor investeerders.

In [6]:
df_price = make_dataset.get_price()

2022-02-09 09:58:44.037 | INFO     | src.data.make_dataset:get_price:120 - Load bond price data
2022-02-09 09:58:44.039 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\price.csv


In [7]:
df_price = make_dataset.impute_price(df_price)

2022-02-09 09:58:44.569 | INFO     | src.data.make_dataset:impute_price:145 - Impute bond price


In [8]:
df_price.head()

,reference_identifier,ccy,rate_dt,mid
0,DE0001135143,EUR,2010-12-17,136.760
1,NL0000102275,EUR,2010-12-17,103.390
2,DE0001135424,EUR,2010-12-17,95.453
3,NL0009446418,EUR,2010-12-17,102.690
4,NL0000102234,EUR,2010-12-17,106.220


In [9]:
df_price.describe()

,mid
count,223766.000000
mean,110.799662
std,15.314964
min,84.429000
25%,102.138000
50%,106.278000
75%,112.223000
max,195.749000


In [10]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223766 entries, 0 to 235985
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reference_identifier  223766 non-null  string        
 1   ccy                   223766 non-null  string        
 2   rate_dt               223766 non-null  datetime64[ns]
 3   mid                   223766 non-null  float64       
dtypes: datetime64[ns](1), float64(1), string(2)
memory usage: 8.5 MB


In [11]:
make_dataset.save_pkl('price', df_price)

2022-02-09 09:58:45.383 | INFO     | src.data.make_dataset:save_pkl:344 - Save preprocessed price data


# Government Yield curves

Governement Yield curves zijn samengesteld uit bondprijzen. de government bond prijs wordt samengesteld door uit te gaan van verschillende referentie bonds met een verschillende looptijd.  Deze yield curvce worden in de markt veel gebruikt voor het berekenen van de intrinsieke waarde. 

In [12]:
df_yield = make_dataset.get_yield()

2022-02-09 09:58:49.401 | INFO     | src.data.make_dataset:get_yield:162 - Load goverment yield curve data
2022-02-09 09:58:49.402 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\yield.csv


In [13]:
df_yield = make_dataset.impute_yield(df_yield)

2022-02-09 09:58:50.541 | INFO     | src.data.make_dataset:impute_yield:199 - Impute yield curve


In [14]:
df_yield.tail()

,country,rate_dt,timeband,ratename,ccy,actual_dt,datedays,bid,offer,int_basis,time,mid
117793,Spain,2022-01-07,5 YEARS,GOV Yield Curve ES BB,EUR,2027-01-07,1830,0.026,0.013,ANNUAL,1826,0.0195
117795,Spain,2022-01-07,6 YEARS,GOV Yield Curve ES BB,EUR,2028-01-07,2195,0.097,0.079,ANNUAL,2191,0.0880
117796,Spain,2022-01-07,7 YEARS,GOV Yield Curve ES BB,EUR,2029-01-07,2561,0.175,0.158,ANNUAL,2557,0.1665
117797,Spain,2022-01-07,8 YEARS,GOV Yield Curve ES BB,EUR,2030-01-07,2926,0.332,0.319,ANNUAL,2922,0.3255
117799,Spain,2022-01-07,9 YEARS,GOV Yield Curve ES BB,EUR,2031-01-07,3293,0.489,0.477,ANNUAL,3287,0.4830


In [15]:
df_yield.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90506 entries, 1 to 117799
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   country    90506 non-null  string        
 1   rate_dt    90506 non-null  datetime64[ns]
 2   timeband   90506 non-null  string        
 3   ratename   90506 non-null  string        
 4   ccy        90506 non-null  string        
 5   actual_dt  90506 non-null  datetime64[ns]
 6   datedays   90506 non-null  int64         
 7   bid        90506 non-null  float64       
 8   offer      90506 non-null  float64       
 9   int_basis  90506 non-null  string        
 10  time       90506 non-null  int64         
 11  mid        90506 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(2), string(5)
memory usage: 9.0 MB


In [16]:
make_dataset.save_pkl('yield', df_yield)

2022-02-09 09:58:51.574 | INFO     | src.data.make_dataset:save_pkl:344 - Save preprocessed yield data


# Inflation data


Als maatstaf voor de inflatie nemen we de inflation linked swap curve. Een belangrijke reden om voor dit inflatiecijfer te kiezen is dat deze data op dagbasis beschikbaar is. 

In [17]:
df_inflation = make_dataset.get_inflation()  

2022-02-09 09:58:55.090 | INFO     | src.data.make_dataset:get_inflation:231 - Load goverment yield curve data
2022-02-09 09:58:55.092 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\DE Inflation.csv
2022-02-09 09:58:55.111 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\FR Inflation.csv
2022-02-09 09:58:55.130 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\ES Inflation.csv
2022-02-09 09:58:55.148 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\IT Inflation.csv
2022-02-09 09:58:55.170 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\US Inflation.csv


In [18]:
df_inflation = make_dataset.impute_inflation(df_inflation)

2022-02-09 09:59:13.688 | INFO     | src.data.make_dataset:impute_inflation:276 - Impute inflation curve


In [19]:
df_inflation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215007 entries, 0 to 216873
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   country    215007 non-null  string        
 1   rate_dt    215007 non-null  datetime64[ns]
 2   timeband   215007 non-null  string        
 3   inflation  215007 non-null  float64       
 4   ratename   215007 non-null  string        
 5   actual_dt  215007 non-null  datetime64[ns]
 6   time       215007 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(1), string(3)
memory usage: 13.1 MB


In [20]:
df_inflation.head()

,country,rate_dt,timeband,inflation,ratename,actual_dt,time
0,Germany,2021-12-23,1 YEAR,3.28625,Inflation,2022-12-23,365
1,Germany,2021-12-22,1 YEAR,3.33875,Inflation,2022-12-22,365
2,Germany,2021-12-21,1 YEAR,3.15625,Inflation,2022-12-21,365
3,Germany,2021-12-20,1 YEAR,3.01375,Inflation,2022-12-20,365
4,Germany,2021-12-17,1 YEAR,2.89875,Inflation,2022-12-17,365


In [21]:
make_dataset.save_pkl('inflation', df_inflation)

2022-02-09 09:59:14.944 | INFO     | src.data.make_dataset:save_pkl:344 - Save preprocessed inflation data


In [22]:
df_bp = join_data.join_price(df_bonds,df_price )
df_bp = build_features.add_duration(df_bp)
make_dataset.save_pkl('bp', df_bp)


2022-02-09 09:59:21.815 | INFO     | src.features.build_features:add_duration:8 - Add remaining duration...
2022-02-09 09:59:21.833 | INFO     | src.data.make_dataset:save_pkl:344 - Save preprocessed bp data


Om de termspread (het verschil tussen de korte en de lange rente) te kunnen berekenen voegen we de government yield data toe. 

In [23]:
df_bpy = join_data.join_yield(df_bp, df_yield)
df_bpy = build_features.add_term_spread(df_bpy)
df_bpy = build_features.add_bid_offer_spread(df_bpy)
make_dataset.save_pkl('bpy', df_bpy)

2022-02-09 09:59:37.000 | INFO     | src.features.build_features:add_term_spread:17 - Add term spread...
2022-02-09 09:59:37.005 | INFO     | src.features.build_features:add_bid_offer_spread:25 - Add bid offer spread...
2022-02-09 09:59:37.272 | INFO     | src.data.make_dataset:save_pkl:344 - Save preprocessed bpy data


In [24]:
df_isin = make_dataset.make_isin(df_bp, df_yield, df_inflation)

2022-02-09 09:59:49.865 | INFO     | src.data.make_dataset:make_isin:296 - Create dataset for bond NL0011220108
2022-02-09 09:59:50.324 | INFO     | src.features.build_features:add_term_spread:17 - Add term spread...
2022-02-09 09:59:50.327 | INFO     | src.features.build_features:add_bid_offer_spread:25 - Add bid offer spread...


Alle data voorbereidingsstappen zijn samengevoegd in onderstaande routine.

(dit duurt ongeveer 1min, 25sec)

In [25]:
make_dataset.make_data()

2022-02-09 09:59:50.509 | INFO     | src.data.make_dataset:get_bond_data:31 - Load bond data
2022-02-09 09:59:50.510 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\bonds.csv
2022-02-09 09:59:50.530 | INFO     | src.data.make_dataset:impute_bonds:82 - Impute bond data
2022-02-09 09:59:50.541 | INFO     | src.data.make_dataset:save_pkl:344 - Save preprocessed bonds data
2022-02-09 09:59:50.586 | INFO     | src.data.make_dataset:get_price:120 - Load bond price data
2022-02-09 09:59:50.587 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\price.csv
2022-02-09 09:59:50.997 | INFO     | src.data.make_dataset:impute_price:145 - Impute bond price
2022-02-09 09:59:51.061 | INFO     | src.data.make_dataset:save_pkl:344 - Save preprocessed price data
2022-02-09 09:59:54.763 | INFO     | src.data.make_dataset:get_yield:162 - Load goverment yield curve data
2022-02-09 09:59:54.764 | INFO     | src.data.make_dataset:read_csv:19 - Loading d